# Multiprocessing

Have you filled out my subscriber survey? If not, please do — it helps me to keep this newsletter relevant. And it only takes a few minutes! Click here to fill it out! 
Last time, we started to look at the "multiprocessing" module, a standard part of Python's standard library. The point of "multiprocessing" is to give us an interface that looks and feels as though we're using threads, even though we're actually using processes.  Not only does this free us from the constraints of the GIL (global interpreter lock), but it also removes the issues of thread safety.

When we're working with processes, we know that our variables are running in their own containers. They cannot affect what is happening in other processes, knowingly or unknowingly, and they also cannot be affected by those other processes.

This raises a question, though: One of the advantages of threads, despite their many pitfalls, is that each thread can contribute to the creation of a larger data structure, or a more complex calculation. If we are working with processes, rather than with threads, how can we get that cooperation to work?

Let's take a trivial example: I for each of the numbers 0-9, I want to get the sum of all numbers from 0 to that number.  In other words, I want to get 0 (0), 1 (0+1), 3 (0+1+2), 6 (0+1+2+3), 10 (0+1+2+3+4), and so forth, all the way up to 45 (0+1+2+3+4+5+6+7+8+9). For now, I just want to calculate these sums, and print them after doing so.  The following program will do so for me: 


In [1]:

#!/usr/bin/env python3

def mysum(numbers):
    total = 0
    for one_number in numbers:
        total += one_number
    return total

if __name__ == '__main__':
    for i in range(10):
        print(i, mysum(range(i)))


0 0
1 0
2 1
3 3
4 6
5 10
6 15
7 21
8 28
9 36


What's going on here? Well, we have to remember that we previously invoked "mysum" directly. Now, we're asking the new "Process" object to invoke it for us. This means that we're not printing the result of "mysum", but rather then result of creating and starting a new multiprocessing.Process object -- which is always going to be None.

If we want to know what "mysum" returned, we'll need to have "mysum" send it to us via another channel. With threads, we could have (in theory) a data structure that's common to all of the threads, such as a list. While lists aren't thread-safe, we could put a lock around it, ensuring that only one thread has access to the list at a time.

With processes, we might think that we don't have options. But Python actually provides us with a number of good ones. One of those options is a queue. Wait, didn't we see queues back when we looked at threads? Yes, we did -- queues are guaranteed to be thread-safe, which is great. But even better is that Python provides multiprocessing.Queue, which basically works in the same way -- only it lets processes send data. Let's see what happens now: 


In [ ]:

#!/usr/bin/env python3

import multiprocessing
from multiprocessing import Queue

q = Queue()

def mysum(i, numbers):
    total = 0
    for one_number in numbers:
        total += one_number
    q.put((i, total))

if __name__ == '__main__':
    processes = []
    for i in range(10):
        p = multiprocessing.Process(target=mysum, args=(i, range(i)))
        processes.append(p)
        p.start()

    for one_process in processes:
        p.join()

    while True:
        print(q.get())

As you can see, I've divided this program into two parts: The first is a function, "mysum", which is basically the same as the "sum" function that is in the Python "builtins" namespace. The "mysum" function takes a sequence of numbers, adding them together one at a time, and then returning the total.

We then invoke "mysum" ten times, once for each of the numbers 0 through 9, passing it a "range" object. We print both the number we're using (i), and the sum of that range.

This actually works just fine: 

Now, I realize that summing numbers is a trivial task, one that modern computers can do many millions of times each second. But let's just suppose that I worry that adding these numbers will take a while, or that the function I'm running is more complex than "mysum".  I might want each run of the function to take place in its own process, such that I can take advantage of multiple CPUs.  Here's how that might look: 


Notice that something important has changed here: No longer are we running the function itself. Rather, we're creating a new instance of multiprocessing.Process, telling it that we want to run "mysum" and giving it the appropriate argument to do so. We then take that new instance of multiprocessing.Process and invoke the "start" method on it, so that it runs.

By the way, if you're wondering why I'm sticking the "if __name__ == '__main__'" code in here, it's to support people who use Windows. Apparently, "multiprocessing" does all sorts of tricks in order to work, and if you don't have this line, that's bad news. 

The good news is that the program runs. The bad news is that it gives us results that we probably won't like: 

In [ ]:

#!/usr/bin/env python3

import multiprocessing

def mysum(numbers):
    total = 0
    for one_number in numbers:
        total += one_number
    return total

if __name__ == '__main__':
    for i in range(10):
        print(i, multiprocessing.Process(target=mysum, args=(range(i),)).start())

Now things are getting a bit more complex: First of all, our "mysum" now takes two arguments -- the value of "i", the current iteration, and also the range object for that iteration. Instead of returning the total, we'll now put a two-element tuple on the "q" queue. We can "put" any object we want (within reason) on that queue, and it'll wait until someone retrieves it.

However, we don't want to retrieve the values right after creating all of those processes. That's because it takes time (if only a bit) for each process to calculate and then return its values. So we first put all of the processes in a list, and then iterate over that list, guaranteeing that each of the subprocesses has finished.

We then use an infinite "while True" loop to retrieve and print each of the tuples we got from "mysum".

The good news: We get and print all of the tuples, as before.

The bad news: The program hangs with an infinite loop.  After all, as we have an infinite loop, from which we never return. So our event loop is ready for whenever we put more data in the queue. Given that we're at the end of the program, though, that's not going to happen, and thus we hang.

This means that we'll need to be a bit more clever about checking that the queue is empty: 

In [ ]:
#!/usr/bin/env python3

import multiprocessing
from multiprocessing import Queue

q = Queue()

def mysum(i, numbers):
    total = 0
    for one_number in numbers:
        total += one_number
    q.put((i, total))

if __name__ == '__main__':
    processes = []
    for i in range(10):
        p = multiprocessing.Process(target=mysum, args=(i, range(i)))
        processes.append(p)
        p.start()

    for one_process in processes:
        p.join()

    while not q.empty():
        print(q.get())

This code is identical to the previous version, except that I have modified the infinite "while True" loop to instead only loop so long as the queue contains something. We know that the processes have all finished by this point, which means that no more data should be put into the queue for now -- which means that when the queue is empty, we have retrieved all of the data.

Moreover: This is clearly a trivial calculation, one which would have given us roughly equal time with threads or processes. However, if the calculations are complex and CPU-bound (i.e., limited by raw calculating power, not because of disk or network slowness), then processes will clearly do better.

Next time, we'll look a some additional ways that our processes can use to communicate with one another.

Until then,

Reuven 
PS: Follow me on Twitter for even more updates about Python, programming, podcasts, and other topics I write about. Follow me at reuvenmlerner. 

